## Backtesting Sesion 1
### Generación y Evaluación de Señales
Este cuaderno utiliza las señales de entrada/salida del oscilador estocástico para
mostrar la evaluación histórica de la estrategia

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import itertools

____
### Datos 
Asumimos que tenemos un dataset disponible con datos de mercado.
Partimos de un snapshot de acciones del IBEX35

In [ ]:
import pickle
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

La variable stock_data es un diccionario {ticker: dataframe} donde ticker es el 
identificador de la acción y dataframe tiene las series OLHCV de cada acción

In [ ]:
stock_data.keys()

Cada dataframe tiene la series de precios

In [ ]:
stock_data['TEF'].head()

____
### Supuestos
En esta primera aproximación asumiremos que
- Estamos observando los precios de cierre diario
- Operamos al día siguiente al precio del próximo cierre

In [ ]:
close_dict = {tk: df.close for tk,df in stock_data.items()}
stock_close = pd.DataFrame(close_dict)
stock_close.head()

Trabajaremos primero con un solo valor, que podemos ir variando

In [ ]:
ticker = 'TEF'
stock_series = stock_close[ticker].dropna()

In [ ]:
stock_series.plot()

___
### Oscilador Estocástico
Es un indicador técnico que construye a partir de una serie de precios,
un valor que expresa la **posición relativa** del precio respecto a una 
**ventana temporal reciente**. Algunas características:
- Se mueve en el intervalo [0, 1] o porcentualmente de 0 a 100
- Es equivalente a realizar una normalización de rango una vez determinada la ventana temporal


In [ ]:
def min_max_scale(s):
    """ Calcula la normalización de rango """
    return (s[-1] - np.min(s)) / (np.max(s) - np.min(s))

In [ ]:
estocastico = stock_series.rolling(window=20).apply(min_max_scale)
estocastico

In [ ]:
# ultimas 500 observaciones
estocastico.iloc[-500:].plot()

___
La serie de base del estocástico es muy errática, por lo que se suele utilizar una indicador suavizado
haciendo una media móvil de los ultimos $k$ días.  Para el cáclulo de las señales de trading se utilizan
diferentes variantes que combinan el estocástico base con el indicador suavizado, o diferentes variantes
a partir de los parámetros que se pueden configurar. En este ejemplo utilizaremos.
- una ventana de tamaño configurable
- solo el indicador suavizado, con una media móvill calculada sobre $k$ igual a un quinto de la ventana 


In [ ]:
def stochastic_osc(s, win):
    """ Calcula la serie primaria del estocástico"""
    lag = int(np.round(win/5))
    so_raw = s.rolling(window=win).apply(min_max_scale, raw=True)
    so = so_raw.rolling(window=lag).mean()
    return so  

In [ ]:
sto_ind = stochastic_osc(stock_series, 20)

# Vemos la serie base y el indicador suavizado
df_sto = pd.DataFrame({
    'estocastico': estocastico,
    'sto_suave': sto_ind
})

In [ ]:
fig, ax = plt.subplots(figsize=(20,4))
df_sto.iloc[-500:].plot(ax=ax)

_____
### Oscilador Estocástico y Señales de Trading
Tradicionalmente el oscilador estocástico se ha utilizado para indicar regiones de **sobrecompra** (ej. 0.8) o **sobreventa** (ej. 0.2).  Las salidas de estas regiones
indicarían un cambio de tendencia en el precio que puede utilizarse como señal de trading de la siguiente forma:
- Si el activo está sobre-vendido y el indicador corta el umbral al alza --> Comprar
- Si el activo está sobre-comprado y el indicador corta el umbral a la baja --> Vender

In [ ]:
def stochastic_osc_states(s, win=14, obought=0.8, osold=0.2):
    """En función de las señales de trading calcula una serie de "estados"
    para indicar que se está 
    1: dentro o invertido
    0: fuera o desinvertido 
    Las señales de trading se determinan a partir 
    de los umbrales de sobre-(compra o venta) """
    
    states = pd.Series(np.zeros(s.shape[0]), index=s.index)
    so = stochastic_osc(s, win)
    
    # Condicion inicial 
    if so.iloc[0] > osold and so.iloc[0] < obought:
        states.iloc[0] = 1
        curr = 1
    else:
        curr = 0
    
    for i in range(s.shape[0] - 2):
        # corta umbral de sobreventa al alza
        if so.iloc[i] < osold and so.iloc[i+1] >= osold:
            curr = 1
        # corta umbral de sobrecompra a la baja
        elif so.iloc[i] > obought and so.iloc[i+1] <= obought: 
            curr = 0
        
        # el cambio de esto ocurre al dia siguiente de la señal
        states.iloc[i+2] = curr
    
    return states

In [ ]:
stock_invested = stochastic_osc_states(stock_series, win=20, obought=0.8, osold=0.2)
stock_invested.iloc[-500:-460]

In [ ]:
# dias invertido/desinvertido
stock_invested.value_counts()

____
Hacemos la gráfica del indicador mostrando los momentos en los que se está invertido
(ultimos 500 dias)

In [ ]:
sto_show = sto_ind.iloc[-500:]
states_show = stock_invested.reindex(sto_show.index)

In [ ]:
fig, ax = plt.subplots(figsize=(20,4))
sto_show.plot(ax=ax)

ax.axhline(0.2, c='g')
ax.axhline(0.8, c='r')
for i in range(sto_show.shape[0]-1):
    if states_show.iloc[i] == 1:
        ax.axvspan(sto_show.index[i], sto_show.index[i+1], facecolor='g', alpha=0.2)

___

### Verificación de Entrada/Salida
Verificamos con detenimiento que el rendimiento de los precios de compra venta correspondan
a la composición de los rendimientos diarios que utilizamos

In [ ]:
simple_ret = stock_series.pct_change()
trade_events = stock_invested.diff()
returns_in = stock_invested.shift(1)
check_df = pd.concat([stock_series, sto_ind, stock_invested, trade_events, returns_in, simple_ret], axis=1)
check_df.columns = ['price', 'estocastico', 'days_in', 'events', 'returns_in','day_ret']
check_df.iloc[-500:-460]

In [ ]:
trade_day_rets = simple_ret.loc['2018-09-25':'2018-10-18']
trade_day_rets

In [ ]:
trade_ret = (trade_day_rets.values + 1).prod() - 1
trade_ret

In [ ]:
trade_ret2 = check_df.price.loc['2018-10-18']/check_df.price.loc['2018-09-24'] - 1
trade_ret2

____

In [ ]:
def state_returns(price, states):
    """ Calcula para una serie y unos estados de 
    estar dentro fuera, cual es el retorno total
    correspondiente.
    Debe tener el mismo pd.Index 
    """
    ret = price.pct_change()
    ret.iloc[0] = 0
    
    in_rets = ret * states.shift(1)
    simple_rets = in_rets + 1 
    total_ret = simple_rets.prod() - 1
    return total_ret

In [ ]:
stock_invested

In [ ]:
total_return = state_returns(stock_series, stock_invested)
total_return

In [ ]:
def ann_returns(price, states):
    tot_ret = state_returns(price, states)
    
    init_date = states.index[0]
    end_date = states.index[-1]
    fyears = (end_date - init_date) / pd.Timedelta(days=365, hours=6)
    
    anual_ret = np.power(tot_ret + 1, 1/fyears) - 1  
    return anual_ret

In [ ]:
ann_returns(stock_series, stock_invested)

Calculamos la alternativa de estar siempre invertido en el activo

In [ ]:
hold_invested = pd.Series(
    np.ones(stock_invested.shape[0]),
    index=stock_invested.index
)

In [ ]:
state_returns(stock_series, hold_invested)

In [ ]:
ann_returns(stock_series, hold_invested)

In [ ]:
def backtest_so_returns(vseries, win=20, obought=0.8, osold=0.2):
    f_states = stochastic_osc_states(
        vseries, 
        win=win, 
        obought=obought,
        osold=osold)
    so_return = ann_returns(vseries, f_states)
    return so_return

In [ ]:
backtest_so_returns(stock_series)

____
### Exploración de Parámetros
Podemos explorar diferentes parámetros para ver como varía el rendimiento

In [ ]:
obought_params = [0.70, 0.80, 0.90]
osold_params = [0.10, 0.20, 0.30]
win_params = [20, 30, 50]

In [ ]:
combined_params = list(itertools.product(win_params, obought_params, osold_params))
combined_params[:10]

In [ ]:
result = {}
for iparams in combined_params:
    (w, b, s) = iparams
    result[iparams] = backtest_so_returns(stock_series, win=w, obought=b, osold=s)
    print(w, b, s)

In [ ]:
sresult = pd.Series(result)
print(sresult.idxmax(), sresult.max())

### Observación
Asumir el resultado del mejor parámetro como válido no es correcto, porque lo que hemos hecho es ajustar el parámetro para ir mejorando el resultado. A futuro esta combinación no tiene por qué dar el mismo resultado.

___

### Ejercicios Propuestos
1. Desarrollar una función que determine el rendimiento y los mejores parámetros del oscilador estocástico, a partir de cualquier 
serie de precios, apoyandose en el código de celdas previas
2. Determinar el mejor conjunto de parámetros y el rendimiento para un conjunto de 5 valores del IBEX35, 
ej. ACS, AMS, REP, ITX y SAN tomando los datos a partir de 2010.

